In [5]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import pathlib

import re

In [6]:
OUTPUT_PATH = pathlib.Path('../test')
assert OUTPUT_PATH.is_dir()

2025-07-17 10:58:51,278 | DEBUG | rfi_pipeline.process (blc07_guppi_60055_23754_Arp220_0134.rawspec.0000.h5) (1596800): Done! Took 21.1s

In [9]:
file_times = []
pattern = r'rfi_pipeline\.process \((.+)\) \(\d+\): Done! Took ([\d\.e+]+)s'
with open(OUTPUT_PATH / 'logs' / 'all_logs.log', 'r') as file:
    for line in file:
        m = re.search(pattern, line)
        if m:
            file_times.append((m.group(1), float(m.group(2))))
file_times

[]

Help on class Scanner in module re:

class Scanner(builtins.object)
 |  Scanner(lexicon, flags=0)
 |
 |  Methods defined here:
 |
 |  __init__(self, lexicon, flags=0)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  scan(self, string)
 |
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |
 |  __dict__
 |      dictionary for instance variables
 |
 |  __weakref__
 |      list of weak references to the object

